In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Author : Lavi Nigam, ML Engineering @ Google
# Linkedin: https://www.linkedin.com/in/lavinigam/

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/lavinigam-gcp/AI-ML-Patterns/blob/main/Notebooks/GA4_Audience_Segmentation_Notebook_and_Implementation_Guide.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

# E-Commerce Audience Segmentation using using Google Analytics (GA4) Data

Learn how to build a system to create audience segmentation of GA4 e-commerce data using BigQuery ML (BQML).

With recent changes, BigQuery ML can directly access GA4 data, bringing capture app and web data in a single interface. This integration opens many opportunities for various machine learning use cases and potential customers. For example, the e-commerce industry can funnel their GA4 data to BQML and expand their analytics with ML capabilities. This pattern aims to help such companies leverage different ML algorithms and scale their analytics with BQ.

Customer segmentation, or grouping customers based on common characteristics, allows e-commerce businesses to provide relevant and timely customer promotions and offers. For example, customers buying behaviors are influenced by their demographics, surfing habits, interests, and even the gadgets they use. The buying behavior will impact what they buy, why they buy, and how much money they spend on each purchase.

When combined with behavioral data, customer segmentation allows online retailers to provide tailored experiences similar to those found in a customer's favorite neighborhood store.

Customers can use k-means clustering, included in BQML, on their e-commerce GA4 data to quickly create customer segments. Clustering will allow users of similar behaviors in GA4 to be segmented, personalizing client outreach by adapting ads and other messages to their tastes and habits, as evidenced by the cluster each customer belongs.

One thing to note is that GA4 has an [built-in predictive audience feature](https://support.google.com/analytics/answer/9805833) that helps segment customers based on churn, purchases, and spending. However, the solution discussed in the pattern is different from that in the sense that it tries to bring more dynamic features like user demographic and interests (page view based). The inbuilt feature is more static, while the pattern is more dynamic since it uses machine learning.

## Objective

The design pattern involves the following steps:

- Loading the e-commerce data
- Exploratory analysis of the data
- Feature engineering and pre-processing of the data for model
- Create, train, and deploy the model in BigQuery
- Evaluate the model to understand various clusters
- Batch prediction using the trained model

## Audience
The pattern is intended for marketing analytics teams for an enterprise, or, teams explicitly responsible for analyzing Google Analytics data. It assumes that you have basic knowledge of the following:

- Machine learning concepts
- Standard SQL & Python

## Costs
This tutorial uses the following billable components of Google Cloud:

- BigQuery
- BigQuery ML
- Cloud Storage


To generate a cost estimate based on your projected usage, use the pricing calculator.

Learn about
- [BigQuery
pricing](https://cloud.google.com/bigquery/pricing),
- [BigQuery ML pricing](https://cloud.google.com/bigquery-ml/pricing),
- [Cloud Storage
pricing](https://cloud.google.com/storage/pricing),

and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## The Dataset
The solution uses the public [GA4 Google Merchandise Store](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=ga4_obfuscated_sample_ecommerce) dataset.

Google Merchandise Store is an online store that sells Google-branded merchandise. The site uses Google Analytics 4's standard web ecommerce implementation along with enhanced measurement. The ga4_obfuscated_sample_ecommerce dataset available through the BigQuery Public Datasets program and contains a sample of obfuscated BigQuery event export data for three months from 2020-11-01 to 2021-01-31.


This dataset contains obfuscated data that emulates what a real world dataset would look like from an actual Google Analytics 4 implementation. Certain fields will contain placeholder values including <Other>, NULL, and " " . Due to obfuscation, internal consistency of the dataset might be somewhat limited.


You can check the schema details of the dataset [here](https://support.google.com/analytics/answer/7029846#zippy=)

There are total 23 columns in the datasets with mixed datatypes, and approximately 4 million rows (each day event is seperate table in the data and total 92 events(tables) are present).

## Exporting Google Analytics data to BigQuery
If instead of the sample data, you want to use your own data from a GA4 property, you can follow the instructions in [(GA4) Set up BigQuery ](https://support.google.com/analytics/answer/9823238#zippy=%2Cin-this-article) Export to export your data.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Python 3
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as {plotly.express, pandas, google.cloud}. Use the latest major GA version of each package.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:s
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q google-cloud-bigquery db-dtypes

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).


3. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

4. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = ""  # @param {type:"string"}

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [ ]:
# Importing some important libraries that will be used during the notebook
import pandas as pd
import plotly.express as px
from google.cloud import bigquery

In [ ]:
# Client manages connections to the BigQuery API and helps
# bundle configuration (project, credentials) needed for API requests.
client = bigquery.Client(PROJECT_ID)

# to make sure all columns are displayed while working with dataframe
pd.set_option("display.max_columns", None)

## Assumptions

## Exploratory Data Analysis (EDA)

You can start by defining some essential variables that can change according to your data. It is always better to consider the most recent records from your data as features. For this purpose, you can set the START_DATE and END_DATE based on your data recency.

In this case, the date range is set for 3 months.

In [ ]:
# Dataset (GA4 Google Merchandise Store) specific Variable
# Change it to your dataset spefic values, if you want to use the code for your data.
# We assume table names will be "events_*"

PROJECT_ID_DATA = "bigquery-public-data"
DATASET_ID_DATA = "ga4_obfuscated_sample_ecommerce"  # ga4-bq-pattern.1crdata.fake_ga4 #ga4_obfuscated_sample_ecommerce
START_DATE = "20201101"
END_DATE = "20210131"  # taking 3 months recent data.

# In queries, these variables are editable so that you can put your project, dataset, and date,
# making it easier for you to make the least amount of changes. Of course, you don't need to change
# it for public data. But, for making the queries editable, it made sense to define them here.
# You can run the whole notebook (mostly) with your data by changing values here.

### Why EDA?

Before starting the audience segmentation, it's essential to identify the various attributes commonly referred to as features (in ML),  based on which the intended audience should be segmented. For example, you can segment your customers based on different languages, locations, pages, and interests on your website, mobile company, shipping methods, etc.

So, with exploratory data analysis (EDA), the goal here is to:

- Find the value distribution, availability, and data types of columns.
- Aggregate summary statistics for important columns.
- Identify critical columns to be used as features.

You can start the data exploration by returning the first five rows of data.
The data has multiple event tables for each day. So, all the tables (events) could be queried by using events* as the wildcard.

[GA4 Data Export Schema](https://support.google.com/analytics/answer/7029846#zippy=)

Note: BigQuery export, by default, are [date sharded tables](https://cloud.google.com/bigquery/docs/partitioned-tables#dt_partition_shard)

In [ ]:
query = f"""
SELECT
  *
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events*`
LIMIT
  5
"""
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data.head()

The first five rows of data can help you understand the tables' composite structure of data types. For example, you can see numerical, categorical, arrays, and struct as data types.

By looking at some columns, you can also identify a few essential features like event_date, event_name, user_ltv, device, geo, traffic_source, platform, and items. However, as discussed earlier, you still are not aware of their value distribution, availability, and data types.

You can check the data types of each column using [INFORMATION_SCHEMA](https://cloud.google.com/bigquery/docs/information-schema-tables) table. It can give you detailed metadata of your columns.

In [ ]:
query = f"""
SELECT
  DISTINCT(column_name),
  data_type
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.INFORMATION_SCHEMA.COLUMNS`
"""

query_job = client.query(query)
predict_data = query_job.to_dataframe()
predict_data

You can start by understanding overall data by getting a quick summary of the data, namely - total events  (event_count), total users (user_count), total days in the data (day_count), and total registered users of the platform (registered_user_id).

In [ ]:
query = f"""

SELECT
  COUNT(*) AS event_count,
  COUNT(DISTINCT user_pseudo_id) AS user_count,
  COUNT(DISTINCT event_date) AS day_count,
  COUNT(DISTINCT user_id) AS registered_user_id
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events*`
"""
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data

As you can observe, there are roughly 4 million events with close to 270,000 users, stretched along 92 days of activity on the platform.

For simplicity, we will **assume** that all user_pseudo_id are unique and represent a single user.


Some potential columns of interest are already identified in our exploration, like - event_date, event_name, user_ltv, device, geo, traffic_source, platform, and items.

---
Now, you can start by looking into `event_name` distribution.

event_name is a significant column in this dataset. It contains all the events triggered as users interact with the Google Merchandise Store like page_view, scroll (scrolling the page), view_item (viewing specific item), etc. You can refer [here](https://developers.google.com/analytics/devguides/collection/ga4/reference/events) for a more detailed meaning of each event_name.

As you can see, it is easy to visualize grouped data in the notebook.

In [ ]:
query = f"""
SELECT
  event_name,
  COUNT(*) as row_count
FROM
   `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events*`
GROUP BY
  event_name
ORDER BY
  row_count DESC
"""
query_job = client.query(query)
result_df = query_job.to_dataframe()
fig = px.bar(
    result_df, x="row_count", y="event_name", title="Event Name Frequency Distribution"
)
fig.show()

You can observe a great imbalance in the frequency of different event_name(s). The top five events based on frequency:

* page_view - User is viewing a page

* user_engagement - Sessions that last 10 seconds or longer

* scroll - User scrolling through a page

* view_item - some content was shown to the user. You can use this to discover the most popular items.

* session_start - User session after the engagement has been initiated.


The other events don't have too many records and hence would be a challenge to be considered a feature. However, you can also notice that typical purchase events - "add_to_cart", "begin_checkout", "add_shipping_info", "add_payment_info", and "purchase" have a tiny amount of records, indicating that this data doesn't contain too many events where a user has bought something.


So,`page_view` seems to be the best filter for the column `event_name` since it has the highest records and covers users' general browsing behavior. However, you can still leverage `add_to_cart` and `purchase` value for purchase information by simply counting a user's total events for these event types. So, although they will be small, you can include them for demonstration purposes to add a little more diversification in the data for the K-Means Model.

Also, remember that the actual key of `page_view` event_name is available in event_params, and their values are in event_params.values.{int/float/string} in nested array format.

Data references:

[Dimensions & Metrics](https://support.google.com/analytics/topic/11151952?hl=en&ref_topic=9228654)


---

Next, you can explore event_params. There are multiple `event_params`  (shown in image) for each `event_name` ( like page_view) that stores the information for that event. It is a nested array and would require UNNEST function to access its key and different values (string, int, and float).

If you are unsure of the GA4 export schema; please refer [here](https://support.google.com/analytics/answer/3437719?hl=en).

In [ ]:
query = f"""
SELECT
  DISTINCT(ep.key) AS event_param_key,
  COUNT(*) AS count
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events*`,
  UNNEST (event_params) AS ep
WHERE
  event_name = 'page_view'
GROUP BY
  ep.key
ORDER BY
  count DESC
"""
query_job = client.query(query)
result_df = query_job.to_dataframe()

fig = px.bar(
    result_df.head(20),
    x="count",
    y="event_param_key",
    title="Event Parameters in Various Events Frequency Distribution",
)
fig.show()

In this graph, you can observe many `key` are part of event_name or a specific event type (like page_view). Moreover, you can also see their respective distributions, which can help us decide which key's to pick for our features.

Among all, the following three will make sense for our current modeling:
- page_title: page visited by a user.
- campaign: if user event has been triggered by some campaigns.
- engagement_time_msec: user engagement time in millisecond for the event.

They will help us define user behavior on the platform. Like:

- page_title can give us a view into user interest based on their browsing history,
- campaigns can help us identify how users are engaging with the platform, and
- engagement times can help us understand their browsing intensity.

These features, together with others, can help diversify clusters. Remember, the more distinct values you have in features, the better for K-means models to segment users in different clusters.

Also, one thing to note here is that we will stick to one `event_type` for these keys: ' page_view`.

You can pick the others based on your business objectives and requirements, but we will stick with these for simplicity.

---

The next feature to look into is page_title. For all page_view that you filter, which a user is browsing, you can also see the page's title. This information can give you enough detail about their interest. However, a user can browse through thousands of pages, and it will be tough to find any relevant information from all of them at once.

One neat way would be to find out the top most visited pages across all users and then rank each user against those pages. Like, top pages may have a title like - "Apparel," "Bags," "Stationary," etc.

In the below query, you can get the count of top most visited pages and then plot them to see the distribution.


In [ ]:
query = """
SELECT
  DISTINCT (ep.value.string_value) AS page_title,
  COUNT(*) AS page_title_count
FROM
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`,
  UNNEST (event_params) AS ep
WHERE
  ep.key = 'page_title'
  AND event_name = 'page_view'
GROUP BY
  page_title
ORDER BY
  page_title_count DESC
LIMIT
20
"""
query_job = client.query(query)
page_title_result_df = query_job.to_dataframe()

In [ ]:
fig = px.bar(
    page_title_result_df,
    x="page_title_count",
    y="page_title",
    title="Different Page Visited Frequency Distribution",
)
fig.show()

Based on the graph, you can pick four or five titles that can help with audience segmentation by adding that "interest" diversification, like:

*   Apparel | Google Merchandise Store
*   Bags | Lifestyle | Google Merchandise Store
*   Hats | Apparel | Google Merchandise Store
*   Women's | Apparel | Google Merchandise Store

For this pattern, the goal is simple, and hence the focus is only to pick the top 4 or 5.

In the next Feature Engineering section, we will see a way to automate this such that you wouldn't have to do this manually.

## Feature Engineering

Now that you have done some basic exploration of GA4 data, you can create different features for audience segmentation.

However, before doing that, you should create a Dataset in BQ Console named "ga4_ecomm_feature_set" inside your project. Then, you can create a table for different kinds of features and store in the dataset.


This will help retain the features for later purposes.

In [ ]:
# You can create the dataset through code.
DATASET_NAME = "ga4_ecomm_feature_set"

try:
    dataset = client.create_dataset(DATASET_NAME, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
except Exception as e:
    print(e)

---


The goal for feature engineering for this pattern is to create four types of features for each user:

1) page_title -> top browsing interest like Apparel_visit_count, Shopping_Cart_visit_count, Mens_visit_count, YouTube_visit_count and etc.

2) page_view -> total browsing events

3) purchase event (add_to_cart, purchase) -> total count of add_to_cart and purchase event

4) user behavior -> aggregated values of user behaviors like; favorite browser, language, device, etc., or most used country, currency, and engagement time.

page_title contains the page's title viewed during the page_view (browsing) event. For example, the user might be visiting the "Apparel" page or "Bags" page.

However, there can be thousands of products and their respective page_tile in a data; hence to simplify, you can only look into the top viewed page and check users' frequency for those specific pages.

You can also include an exclude list while querying the top pages based on your N value.

In [ ]:
exclude_list = ("Home", "Google Online Store")
top_n = 20
# query to get page_title frequency for all pages visited.
query = f"""
WITH
  top_viewed_pages AS (
  SELECT
    DISTINCT (ep.value.string_value) AS page_title,
    REGEXP_REPLACE(REPLACE(TRIM(SPLIT(ep.value.string_value, "|")[SAFE_OFFSET(0)]), " ", "_"), r'[^a-zA-Z]', '') AS key,
    COUNT(*) AS page_title_count,
  FROM
    `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
    UNNEST (event_params) AS ep
  WHERE
    ep.key = 'page_title'
  GROUP BY
    page_title
  ORDER BY
    page_title_count DESC
  LIMIT
    {top_n} )
SELECT
  *
FROM
  top_viewed_pages
WHERE
  page_title NOT IN {exclude_list}
"""
query_job = client.query(query)
df = query_job.to_dataframe()
df

In the Google Merchandise Store, these are the top most visited pages cumulatively by all users. This table might look very different for your dataset and store.
However, since we are using the general function, it won't matter as our goal is to identify the topmost pages. This query and function will give you the top-visited pages as long as you have page_title in your export schema.

The goal is to create a dynamic query such that it can take top pages visited across all users and then count the number of times the user has seen that page to generate user interest. However, the page_title text is sometimes very descriptive and may require some cleaning before being used for the dynamic query.

In [ ]:
def get_query_string(page_title_keys, page_title):
    """
    The function accepts:
    page_title_keys: All the keys from page_title (extracted from the previous function) that are used as column names and variable names.
    page_title: page_title dataframe that contains all columns

    This function achieves two things:
    1) Dynamically generates the WHEN matching query
    2) Dynamically generates the SELECT query

    case_when_string_list: Empty list that will store CASE WHEN dynamic SQL query string based on page_title keys (column names)
    select_key_string_list: Empty list that will store SELECT dynamic SQL query string based on page_title keys (column names)
    """
    case_when_string_list = []
    select_key_string_list = []
    for eachkey in page_title_keys:
        select_key_string = f"IFNULL(item_page_view_table.pageVisit_count_sum_{eachkey},0) AS {eachkey}_visit_count,\n"
        select_key_string_list.append(select_key_string)

    for index, row in page_title.iterrows():
        case_when_string = f"""WHEN ep.value.string_value LIKE "{row['page_title']}" THEN '{row['key']}'\n"""
        case_when_string_list.append(case_when_string)
    return (" ".join(select_key_string_list), " ".join(case_when_string_list))

In [ ]:
(select_key_string, case_when_string) = get_query_string(list(df["key"]), df)

In [ ]:
print(select_key_string)
# dynamic select query based on user selected pages

In [ ]:
print(case_when_string)
# dynamic case when query based on user selected pages.

In [ ]:
rl

In [ ]:
Hey Carlosquery = f"""
CREATE OR REPLACE TABLE
  ga4_ecomm_feature_set.ga4_features AS
SELECT
  item_page_view_table.user_pseudo_id,
  {select_key_string}
  total_event_table.total_event_count,
  IFNULL(c.count_add_to_cart_count,
    0) AS count_add_to_cart,
  IFNULL(d.count_purchase_count,
    0) AS count_purchase,
  IFNULL(browse_feature.favorite_device_medium,
    'NotAvailable') AS favorite_device_medium,
  IFNULL(browse_feature.favorite_mobile_company_name,
    'NotAvailable') AS favorite_mobile_company_name,
  IFNULL(browse_feature.favorite_lang,
    'NotAvailable') AS favorite_lang,
  IFNULL(browse_feature.most_used_country,
    'NotAvailable') AS most_used_country,
  IFNULL(browse_feature.most_used_campaign,
    'NotAvailable') AS most_used_campaign,
  IFNULL(browse_feature.average_engagement_time_minute,
    0) AS average_engagement_time_minute,
FROM (
  SELECT
    *
  FROM (
    SELECT
      user_pseudo_id,
      CASE {case_when_string}
      ELSE
      "Others"
    END
      AS page_title,
      COUNT(ep.value.string_value) AS pageVisit_count
    FROM
      `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
      UNNEST(event_params) AS ep
    WHERE
      ep.key = 'page_title'
      AND ep.value.string_value IN {tuple(df['page_title'])}
      AND event_date BETWEEN '{START_DATE}'
      AND '{END_DATE}'
      AND event_name = 'page_view'
    GROUP BY
      user_pseudo_id,
      page_title ) PIVOT ( SUM(pageVisit_count) AS pageVisit_count_sum FOR page_title IN {tuple(df['key'])} ) ) item_page_view_table
LEFT JOIN (
  SELECT
    user_pseudo_id,
    COUNT(DISTINCT(event_date)) AS total_event_count,
  FROM
    `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`
  WHERE
    event_date BETWEEN '{START_DATE}'
    AND '{END_DATE}'
  GROUP BY
    user_pseudo_id) total_event_table
ON
  item_page_view_table.user_pseudo_id = total_event_table.user_pseudo_id
LEFT JOIN (
    SELECT
        user_pseudo_id,
        COUNT(*) AS count_add_to_cart_count
      FROM
        `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
        UNNEST (event_params) AS ep
      WHERE
        event_name = 'add_to_cart'
        AND
        event_date BETWEEN '{START_DATE}'
    AND '{END_DATE}'
      GROUP BY
        user_pseudo_id) c
    ON
      item_page_view_table.user_pseudo_id = c.user_pseudo_id
LEFT JOIN (
    SELECT
        user_pseudo_id,
        COUNT(*) AS count_purchase_count
      FROM
        `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
        UNNEST (event_params) AS ep
      WHERE
        event_name = 'purchase'
        AND
        event_date BETWEEN '{START_DATE}'
    AND '{END_DATE}'
      GROUP BY
        user_pseudo_id) d
    ON
      item_page_view_table.user_pseudo_id = d.user_pseudo_id
LEFT JOIN (
  SELECT
    user_pseudo_id,
    MAX(device.category) AS favorite_device_medium,
    MAX(device.mobile_brand_name) AS favorite_mobile_company_name,
    MAX(device.LANGUAGE) AS favorite_lang,
    MAX(geo.country) AS most_used_country,
    MAX(
    IF
      (events.key = "campaign",
        events.value.string_value,
        NULL)) AS most_used_campaign,
    ROUND(AVG(
      IF
        (events.key = "engagement_time_msec",
          events.value.int_value,
          NULL))/60000,2) AS average_engagement_time_minute,
  FROM
    `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
    UNNEST(event_params) AS events,
    UNNEST(items) AS item
  WHERE
    event_date BETWEEN '{START_DATE}'
    AND '{END_DATE}'
  GROUP BY
    user_pseudo_id ) browse_feature
ON
  item_page_view_table.user_pseudo_id = browse_feature.user_pseudo_id
WHERE
  item_page_view_table.user_pseudo_id IS NOT NULL
"""
query_job = client.query(query)
# print(query)

In [ ]:
query = """

SELECT
  *
FROM
  `ga4_ecomm_feature_set.ga4_features`
LIMIT
5
"""

query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()

## BQML Modeling

Now that you have the feature data stored in 'ga4_features' table, you can start the BQML K-means Modeling.

You should define dataset_id and feature_table, which can be used while building Python code for modeling.

In [ ]:
dataset_id = "ga4_ecomm_feature_set"
feature_table = "ga4_features"  # table name

Before starting the model, there are some important points to remember:

1) Remove user_pseudo_id, as it is not a feature but an identifier for the user. In addition, this column has high cardinality, meaning a higher amount of unique values, and hence should not be taken during model building. Make sure to remove any such columns (typical examples like - serial. nos, some IDs). Otherwise, model building times will be massive and might not yield good results.

2) The dataset contains both categorical and numerical columns, and BQML will take care of normalizations and categorical encoding automcatically.



---


The most important part of K-means clustering is to decide the number of clusters to be used during model training. The parameter `num_cluster` has to be determined by user. The value chooses how many segments you want to do in your data. Depending on business requirements, it can range from values as low as 3 and can go up to 100 or more.

Some basic examples like hand-written digits clustering will have a straightforward `num_clusters` value as 10, since it's known beforehand.

Similarly, maybe your marketing or business teams would like to run some campaigns against three segments of users. Here, you can also observe that `num_clusters` value is know (k=3).

However, it is not always this straightforward; hence, identifying the correct value for such situations is crucial.

For such situations, we can use the [Hyper Parameter Tuning](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-hyperparameter-tuning) feature of BQML to identify the best value of `num_clusters` based on the [Davis-Bouldin](https://en.wikipedia.org/wiki/Davies%E2%80%93Bouldin_index) score and elbow method.


---

For this experiment, it is important to set a few parameters:
- min_cluster_num - minimum number for clusters

- max_cluster_num - maximum number for clusters

- num_trails - The maximum number of submodels to train. The tuning will stop when num_trials submodels are trained or when the hyperparameter search space is exhausted. The maximum value is 100. Since we have fourteen values to explore, we will set this as 14.

- max_parallel_trails - The maximum number of trials to run at the same time. The default value is one, and the maximum value is 5. We will set this at five the max since there are a lot of parameters to be explored, and hence parallelizing them would make the process faster.


This query will take some time to run, roughly around ~8 minutes for this data. It may vary for your data.

In [ ]:
min_cluster_num = 4
max_cluster_num = 20
model_name = "model_hptune"

model_hp_tunning_query = f"""
CREATE MODEL
  `ga4_ecomm_feature_set.{model_name}`
OPTIONS
  ( MODEL_TYPE='KMEANS',
    NUM_CLUSTERS = hparam_range({min_cluster_num},{max_cluster_num}),
    num_trials=14,
    max_parallel_trials=5
    ) AS
SELECT
  * EXCEPT (user_pseudo_id)
FROM `{PROJECT_ID}.{dataset_id}.{feature_table}`

"""
query_job = client.query(model_hp_tunning_query)

Once the experiment finishes (around ~ 8 minutes), you can extract the Davies Bouldin score using ML.TRAIL_INFO method. These values will be required to identify the best deal.
Ideally, the lowest value of the score gives the best value of `num_clusters`

In [ ]:
ml_evaluate_query = f"""
WITH trial_data as (
select *
    FROM ML.TRIAL_INFO(MODEL ga4_ecomm_feature_set.{model_name})
)
select
trial_id,
hyperparameters.NUM_CLUSTERS,
hparam_tuning_evaluation_metrics.davies_bouldin_index
from
trial_data
"""
query_job = client.query(ml_evaluate_query)
ml_info_df = query_job.to_dataframe()
ml_info_df.sort_values(by="NUM_CLUSTERS", inplace=True)
ml_info_df

However, the lowest value is not always the best indicator. The best value is the value after which the score stabilizes and doesn't drop or increase much. This can be easily figured out using an elbow curve by plotting a line graph of score vs. cluster values.

![](https://media.geeksforgeeks.org/wp-content/uploads/20190606105550/distortion1.png)

In the above example, you can see that we are plotting distortions (David Bouldin Score) vs. K value (`num_clusters`). The goal is to find the "elbow" position of the graph and select that value for K.

In the grapgh, you can see, K=3 or 4 seems to be an "elbow," after which not much reduction in the score is observed, and it stabilizes. This is just an example; now you can draw a similar graph for the trials we have just performed.

In [ ]:
fig = px.line(ml_info_df, x="NUM_CLUSTERS", y=["davies_bouldin_index"], markers=True)
fig.show()

In the graph, you don't get a proper elbow which can mean three things:

1) You need more or better features to get better clusters, or

2) You need to increase the `num_clusters` values in the trial.

3) Data inherently doesn't have clusters and can not be segmented.

You can try out the first two methods. However, be very careful in increasing the K value from 20 since it should have a strong business reason.

Usually, the more the K value, the harder it is to make sense of the underlying features of clusters since there will be a lot of overlaps between values.

In the graph, you can notice that the lowest value of David Bouldin's score is sitting at k=20. However, analyzing those many clusters would be challenging and overlapping, so for simplicity, we keep that at k=5 and assume that data has inherent clusters.

As discussed earlier, this should be in tandem with your business objectives.

In [ ]:
num_cluster = 5
model_name = "model_c5"

model_query = f"""
CREATE OR REPLACE MODEL
  `ga4_ecomm_feature_set.{model_name}`
OPTIONS
  ( MODEL_TYPE='KMEANS',
    NUM_CLUSTERS = {num_cluster},
    KMEANS_INIT_METHOD='KMEANS++'
    ) AS
SELECT
  * EXCEPT (user_pseudo_id)
FROM `{PROJECT_ID}.{dataset_id}.{feature_table}`
"""
query_job = client.query(model_query)

Once the training is complete, model objects are stored in BQ. You can look into the model evaluations through the BQ console that gives you cluster statistics.

For example, the below images split the evaluations into numeric and categorical columns.

![](https://screenshot.googleplex.com/bkMq8m5bvDJVzQL.png)

From the above table, you can observe the total count of records in each cluster and the centroid values (approximately average) of each numerical feature shown in the green bar with value. The values can give you a good insight into the character of each cluster, for example:

- Cluster 1: Group of customers interested in Google Dino Game Tshirts and Youtube Merchandise. The marketing/sales team can use this information later to target these specific customers for any new game t-shirts or YouTube inventory.


This, however, is just an evaluation of the training data. In actual scenarios, you will use this model to predict clusters on new data and then analyze each character in detail.

![](https://screenshot.googleplex.com/CFqy9ezurZPhP7o.png)

![](https://screenshot.googleplex.com/BDVD8ASxAPkFCLL.png)

Similarly above graph is a summary of statistics for categorical data. It presents the count distribution for each category for a given categorical column.

In the example, you can see the graph for two columns, `favorite_mobile_company_name` and `most_used_campaign.`
This can also help you understand some specific characters of each cluster.

For example, in the second graph for `most_used_campign,` cluster 4 is a dominant "referral" customers.

We will explore detailed exploration and predictions in the next section.

The point to note here is that these are not the best clusters. You can already see overlapping values among features and uneven distributions of the count of users. You can again refer to the David Bouldin Score graph and notice that at k=5, we are not at the lowest loss (score).

However, we chose to pick five to keep the analysis simple. In ideal scenarios, you would coordinate with the business and try to balance choosing k values with your score.

## Batch Prediction

Now that the model is finalized, it's time to make batch predictions on new or incoming data. Since we don't have either, we will try to predict the same data (features) we used for modeling.

The most crucial point is that the model prediction will happen on features and not on raw data in ideal scenarios. So, when new or incoming data comes, you need to run the feature engineering section to generate new features that your model can accept for batch predictions.

For simplification and demonstration purposes, we use the same features for predictions used to train the model.

You can use the `ML.PREDICT` method on the features. Here you don't need to exclude user_pseduo_id since that will be required to retarget customers based on their clusters ids.

Another essential thing to do is to save the outputs in a BQ table so that it can be later used for any further integrations.

In [ ]:
prediction_data_table_name = "model_prediction_c5"
query = f"""
CREATE OR REPLACE TABLE
  ga4_ecomm_feature_set.{prediction_data_table_name} AS
SELECT
  * EXCEPT(NEAREST_CENTROIDS_DISTANCE)
FROM
  ML.PREDICT(MODEL ga4_ecomm_feature_set.{model_name},
    (
    SELECT
      *
    FROM
      `{PROJECT_ID}.{dataset_id}.{feature_table}`))
"""
query_job = client.query(query)

In [ ]:
query = f"""
SELECT
  *
FROM
  {PROJECT_ID}.ga4_ecomm_feature_set.{prediction_data_table_name}
LIMIT 5
"""
query_job = client.query(query)
predict_data = query_job.to_dataframe()
predict_data.head()

You can see the table with predictions has a column `CENTROID_ID` that contains their cluster assignment.

## Prediction Analysis

The last step would be to analyze and understand each cluster in more detail. This analysis will help customize triggers to Google Ads Connector later.

This step is also crucial since it helps validate the segmentation modeling from a business point of view and helps pivot if necessary.

To achieve the cluster-specific behavior, you can do a `group by` `CENTROID_ID` and get the weights' average.

The average weight signifies their affinity to "visit" or "browse" specific pages they have visited. This value can help identify clusters of people interested in some particular types of pages, as one of the clusters could be everyone who likes "Hats," etc.


For simplification, we are only trying to segment customers based on their affinity to visit some specific pages like "hats," "apparel," "drinkware," etc.

You can and should include more features based on your business requirements.

In [ ]:
query = f"""
SELECT
  CENTROID_ID,
  COUNT(user_pseudo_id) as total_user,
  AVG(Apparel_visit_count) as avg_apparel_weight,
  AVG(MensUnisex_visit_count) as avg_unisex_weight,
  AVG(Drinkware_visit_count) as avg_drinkware_weight,
  AVG(Bags_visit_count) as avg_bags_weight,
  AVG(Hats_visit_count) as avg_hats_weight,
  AVG(Womens_visit_count) as avg_womens_weight,
  AVG(CampusCollection_visit_count) as avg_campus_weight,
  AVG(Sale_visit_count) as avg_salepage_weight,
  AVG(YouTube_visit_count) as avg_youtubepage_weight,
  AVG(average_engagement_time_minute) as avg_timespent
FROM
  `{PROJECT_ID}.{dataset_id}.{prediction_data_table_name}`
GROUP BY CENTROID_ID
ORDER BY CENTROID_ID
"""
query_job = client.query(query)
predict_data = query_job.to_dataframe()
predict_data.head()

Once you have the average values for each feature (numerical), you can find the most dominant feature where the values are high.

For that, you can take the top 2 values and their feature names for each cluster. You can also write a dynamic python function that takes all the rows and then sorts them to take the topmost values based on N (2 in our case, since we need the top 2 high values).

The N is dominanat_category in the code.


In [ ]:
dominant_category = 2


def check_max(row):
    new_row = row.drop(["total_user", "CENTROID_ID", "avg_timespent"])
    return list(new_row.sort_values(ascending=False).index)[:dominant_category]


predict_data["dominant_categories"] = predict_data.apply(check_max, axis=1)

In [ ]:
predict_data[["CENTROID_ID", "total_user", "dominant_categories"]]

This will give each cluster the most dominant features (based on their highest average weight of page viewing) and total_user in each cluster. Now that you have that, you can explain the characteristics property of each cluster.

- Cluster 1 -> Customers who love bags and are interested in unisex collections.
- Cluster 2 -> Customers who love hats and apparel.
- Cluster 3 -> Customers who love hats but also love unisex collections. This is similar to cluster 2 since both customers love hats. You can combine both clusters (2 & 3) and call customers who love hats.
- Cluster 4 -> Customers who are only interested in unisex collections, and usually visit the sales page. They can be targeted if a sale goes in a unisex collection.
- Cluster 5 -> Customers who are interested in the general apparel section and love the YouTube merchandise. They can be targeted for any new apparel launches or some new merchandise for YouTube.


These overly simplified characteristics assume other features do not affect their browsing behavior. Sometimes, simplification is essential to make the single characteristics stand out and make targeting easier.

This is not 100% correct segments, but they are approximately correct. The goal is not to find 100% accurate clusters but to find some approximation of their behaviors such that some Google Ads targeting can be done in dynamic ways.


---


You can also plot the weights in Heat Maps. Heats maps are easier to identify the dominant features because of the value legend on the right.

For example, Cluster 2 can be seen as dominant with "hats" and "apparel."

In [ ]:
fig = px.imshow(predict_data.drop("total_user", axis=1), text_auto=True)
fig.show()

## Next Steps

Once everything is finalized and your business has given the green light on cluster characteristics, it's time to integrate this further into Google Ads Connector.

The GAC (commonly referred to as Tentacles) can send a massive amount of data to GMP (e.g., Google Analytics, Campaign Manager) or Google Ads automatically and reliably.

In the next part of the series in the GA4 patterns, you will see how to achieve the integration.

First, you will learn to set up Tentacles and then connect it to BQ table where we have the final centroids for all `user_pseudo_ids`.

## Clean Up

In [ ]:
# # Are you sure you want to do this? This is to delete all models
# models = client.list_models(dataset_id) # Make an API request.
# for model in models:
#     full_model_id = f"{model.dataset_id}.{model.model_id}"
#     client.delete_model(full_model_id)  # Make an API request.
#     print(f"Deleted: {full_model_id}")
# # Are you sure you want to do this? This is to delete all tables and views
# tables = client.list_tables(dataset_id)  # Make an API request.
# for table in tables:
#     full_table_id = f"{table.dataset_id}.{table.table_id}"
#     client.delete_table(full_table_id)  # Make an API request.
#     print(f"Deleted: {full_table_id}")